In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import os
from PIL import Image
import numpy as np

# Ana veri seti klasörünüz
dataset_dir = "dataset"
haze_dir = os.path.join(dataset_dir, "haze")
clear_dir = os.path.join(dataset_dir, "clear")


In [20]:
from google.colab import drive
import os
import matplotlib.pyplot as plt
from PIL import Image

# dataset klasörünü bağlama
dataset_path = '/content/drive/MyDrive/dataset'
#os.symlink(dataset_path, '/content/dataset')  # Sembolik bağlantı
working_dir = '/content/dataset'

# Alt klasörler
haze_dir = os.path.join(dataset_path, 'haze')
clear_dir = os.path.join(dataset_path, 'clear')

In [21]:
import os
print("Çalışma dizini:", os.getcwd())

Çalışma dizini: /content


In [22]:
# Haze ve clear klasörlerindeki tüm görüntüleri listeleme
haze_images = [os.path.join(haze_dir, fname) for fname in os.listdir(haze_dir) if fname.endswith(".jpg")]
clear_images = [os.path.join(clear_dir, fname) for fname in os.listdir(clear_dir) if fname.endswith(".jpg")]

# İlk birkaç görüntüyü kontrol etmek için yükleyip gösterelim
for img_path in haze_images[:5]:
    img = Image.open(img_path)
    img.show()


In [23]:
import numpy as np
from PIL import Image
from multiprocessing import Pool

# Görüntüleri yükleme ve ön işleme fonksiyonu
def load_and_preprocess_image(img_path, target_size=(256, 256)):  # Boyutlandırmayı 256'e düşürdüm
    try:
        print(f"İşleniyor: {img_path}")  # Loglama eklendi
        img = Image.open(img_path).resize(target_size)  # Yeniden boyutlandırma
        img = np.array(img) / 255.0  # Normalizasyon
        return img
    except Exception as e:
        print(f"Görüntü yüklenemedi: {img_path}, Hata: {e}")
        return None

# Paralel işleme ile görüntüleri yükleme fonksiyonu
def load_images_parallel(image_paths, num_workers=4):
    with Pool(processes=num_workers) as pool:
        images = pool.map(load_and_preprocess_image, image_paths)

    # None dönen hatalı görüntüleri filtreleme
    images = [img for img in images if img is not None]
    return np.array(images)

# Alt küme ile test etmek isterseniz:
haze_images_subset = haze_images[:50]  # İlk 50 sisli görüntüyü al
clear_images_subset = clear_images[:50]  # İlk 50 temiz görüntüyü al

# Haze görüntüleri yükleme
print("Haze görüntüleri yükleniyor...")
haze_data = load_images_parallel(haze_images_subset, num_workers=4)  # 4 çekirdek kullanılıyor
print(f"Haze görüntüleri yüklendi, toplam {len(haze_data)} adet görüntü işleme alındı.")

# Clear görüntüleri yükleme
print("Clear görüntüleri yükleniyor...")
clear_data = load_images_parallel(clear_images_subset, num_workers=4)
print(f"Clear görüntüleri yüklendi, toplam {len(clear_data)} adet görüntü işleme alındı.")

# Yüklenen veri kümelerinin boyutlarını kontrol et
print(f"Haze veri şekli: {haze_data.shape}")
print(f"Clear veri şekli: {clear_data.shape}")



Haze görüntüleri yükleniyor...
İşleniyor: /content/drive/MyDrive/dataset/haze/8824_0.9_0.06.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8825_0.8_0.1.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8829_1_0.04.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8818_0.95_0.12.jpg



İşleniyor: /content/drive/MyDrive/dataset/haze/8825_0.9_0.2.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8823_0.9_0.08.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8829_0.8_0.06.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8823_0.95_0.06.jpg



İşleniyor: /content/drive/MyDrive/dataset/haze/8818_1_0.04.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8829_0.95_0.12.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8827_0.8_0.04.jpg
İşleniyor: /content/drive/MyDrive/dataset/haze/8823_0.95_0.12.jpg


İşleniyor: /content/drive/MyDrive/dataset/haze/8827_0.8_0.1.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8824_0.8_0.16.jpg
İşleniyor: /content/drive/MyDrive/dataset/haze/8818_0.8_0.12.jp

In [24]:
import os

# Dataset yolunu kontrol et
dataset_path = '/content/drive/MyDrive/dataset'
clear_dir = os.path.join(dataset_path, "clear")
haze_dir = os.path.join(dataset_path, "haze")

# Klasörlerin varlığını doğrula
if not os.path.exists(clear_dir):
    raise FileNotFoundError(f"'clear' klasörü bulunamadı: {clear_dir}")
if not os.path.exists(haze_dir):
    raise FileNotFoundError(f"'haze' klasörü bulunamadı: {haze_dir}")

# Clear klasöründeki tüm dosyaların isimlerini alalım
clear_images = [fname for fname in os.listdir(clear_dir) if fname.endswith(".jpg")]

# Haze klasöründeki tüm dosyaların isimlerini alalım
haze_images = [fname for fname in os.listdir(haze_dir) if fname.endswith(".jpg")]

# Clear görüntüsüne karşılık gelen haze görüntülerini eşleştireceğimiz bir sözlük oluşturuyoruz
haze_images_dict = {clear_img: [] for clear_img in clear_images}

# Clear görüntülerine karşılık gelen haze görüntülerini bulalım
for clear_img in clear_images:
    # Clear görüntüsünün dosya adı (uzantısız)
    base_name = os.path.splitext(clear_img)[0]

    # Bu base_name'e sahip tüm haze dosyalarını buluyoruz
    corresponding_haze = [haze_img for haze_img in haze_images if haze_img.startswith(base_name)]

    # Eğer haze görüntüleri varsa, clear görüntüsüne karşılık gelen listeye ekliyoruz
    if corresponding_haze:
        for haze_img in corresponding_haze:
            haze_images_dict[clear_img].append(os.path.join(haze_dir, haze_img))

# Sonuçları kontrol edelim
for clear_img, haze_list in haze_images_dict.items():
    print(f"{clear_img} için {len(haze_list)} haze görüntüsü bulundu:")
    for haze_img in haze_list:
        print(f"    - {haze_img}")


Görüntülenen çıkış son 5000 satıra kısaltıldı.
    - /content/drive/MyDrive/dataset/haze/3168_0.85_0.06.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.95_0.08.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.8_0.04.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.8_0.16.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.8_0.12.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.8_0.2.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.95_0.04.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.85_0.1.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.85_0.2.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.8_0.06.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.85_0.12.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.8_0.1.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.9_0.04.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.9_0.1.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.9_0.12.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.95_